In [1]:
using Dates

global SEED = Dates.value(Dates.now())  # Use current epoch time as default seed

# using POSIX params for LCG
# https://en.wikipedia.org/wiki/Linear_congruential_generator#Parameters_in_common_use
const A = 25214903917
const C = 11
const MOD = 2^48


function set_seed(seed::Int)
    global SEED = seed
end


function set_user_seed(seed::Int)
    global SEED = seed * 7856209
end


function seed_setter(seed::Union{Int, Nothing}=nothing)
    if !isnothing(seed)
        set_user_seed(seed)  # only set user seed once so we get new seed in subsequent calls
    end
end


function get_seed()
	return SEED
end


function gen_prn()
    seed = get_seed()
    prn = mod(A * seed + C, MOD)
    set_seed(prn)
    return prn
end

gen_prn (generic function with 1 method)

In [81]:
function loop(n)
    x = zeros(n)
    for i in 1:n
        x[i] = gen_prn()  # simple assignment seems faster (at least per benchmarking) than += operations
    end
end


function dot(n)
    x = zeros(n)
    x .= gen_prn.()
end

dot (generic function with 1 method)

In [108]:
n = 100000000

100000000

In [109]:
@time begin
    loop(n)
end

 11.630599 seconds (400.00 M allocations: 6.706 GiB, 8.87% gc time)


In [110]:
@time begin
    dot(n)
end

 11.781914 seconds (400.00 M allocations: 6.706 GiB, 7.06% gc time)


100000000-element Vector{Float64}:
 2.60963012289903e14
 2.4948936354382e13
 2.51344392099649e14
 1.89117080751288e14
 1.11076951521891e14
 4.4037745643058e13
 1.42098424166229e14
 1.26815601798e12
 2.48047862819479e14
 2.85636938838e11
 1.0317808092329e13
 1.23890842866176e14
 6.2086827321867e13
 ⋮
 2.45832672626451e14
 9.084866011677e13
 1.18492515912581e14
 1.3909323490732e13
 3.1017944322631e13
 2.3239030411846e13
 1.26956307065817e14
 2.03786789003632e14
 2.38000926088379e14
 1.28692356185514e14
 2.39941867453805e14
 8.3232276780404e13

In [27]:
using Pkg

Pkg.activate(".")
Pkg.resolve()
Pkg.instantiate()

  Activating environment at `~/Github/RandomVariates.jl/Project.toml`
   Installed BenchmarkTools ─ v1.3.1
    Updating `~/Github/RandomVariates.jl/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1
    Updating `~/Github/RandomVariates.jl/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.1


In [96]:
using BenchmarkTools

In [111]:
n = 1000

1000

In [100]:
b = @benchmarkable loop(n)
tune!(b)
run(b)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   91.219 μs …  25.158 ms  ┊ GC (min … max): 0.00% … 99.57%
 Time  (median):      96.481 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   108.311 μs ± 402.922 μs  ┊ GC (mean ± σ):  6.43% ±  1.72%

  ▂▅▆█▅▇▃▃      ▁                                               ▁
  ████████▇▆▅▅▇▇████▇▆▅▅▆▅▆████▆▇▅▇▇▆▆▇▆▆▇▆▅▆▅▅▃▅▅▆▅▆▆▅▆▄▄▄▄▅▅▆ █
  91.2 μs       Histogram: log(frequency) by time        179 μs <

 Memory estimate: 62.45 KiB, allocs estimate: 3490.

In [101]:
b = @benchmarkable dot(n)
tune!(b)
run(b)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   91.390 μs …  23.690 ms  ┊ GC (min … max): 0.00% … 99.54%
 Time  (median):      96.190 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   106.584 μs ± 392.875 μs  ┊ GC (mean ± σ):  6.37% ±  1.72%

  ▄▅▅█▅▆▂▄▂      ▁                                              ▁
  █████████▇▅▄▇▇███▇▆▅▄▃▃▃▅▄▇▇▇▆▆▆▄▅▆▆▃▃▅▆▆▆▅▅▅▆▆▄▄▄▅▄▆▅▄▄▃▄▄▄▄ █
  91.4 μs       Histogram: log(frequency) by time        174 μs <

 Memory estimate: 62.45 KiB, allocs estimate: 3490.